In [1]:
using Pkg

Pkg.activate(@__DIR__)
Pkg.instantiate()

  Activating project at `c:\Users\ayats\Documents\aeer-parsing`

In [54]:
import CSV as CSV
import DataFrames: DataFrame
import DataFrames as DataFrames
using Printf

header = ["year","day","datetm","min","ws_25","wd_25","tp_25","ws_50","wd_50","tp_50","missing"]
dfs=Any[]

for month = 1:12
    monthstr = @sprintf "%02d" month
    df = CSV.read(
        "./data/ORN09$(monthstr)WD.csv", DataFrame;
        header=header, 
        delim=';',
        dateformat="yyyy-mm-dd HH:MM:SS",
        drop=[:missing]
    )

    push!(dfs, df)
end

result = reduce(vcat, dfs)
result

Row,year,day,datetm,min,ws_25,wd_25,tp_25,ws_50,wd_50,tp_50
,Int64,Int64,DateTime,Int64,Any,Any,Any,Any,Any,Any
1,2009,1,2009-01-01T00:00:00,0,2.241,328.0,24.64,3.571,322.1,25.54
2,2009,1,2009-01-01T00:10:00,10,2.386,351.4,24.29,3.29,342.0,24.96
3,2009,1,2009-01-01T00:20:00,20,1.907,355.1,24.19,2.996,341.3,25.07
4,2009,1,2009-01-01T00:30:00,30,1.113,84.9,24.24,1.613,0.517,25.16
5,2009,1,2009-01-01T00:40:00,40,4.031,173.4,24.76,4.694,177.7,25.18
6,2009,1,2009-01-01T00:50:00,50,1.185,241.9,25.97,1.447,198.3,25.98
7,2009,1,2009-01-01T01:00:00,60,1.727,56.83,25.16,2.488,90.4,25.04
8,2009,1,2009-01-01T01:10:00,70,2.053,32.83,25.48,1.88,49.17,25.78
9,2009,1,2009-01-01T01:20:00,80,1.76,27.14,25.32,2.135,47.11,25.96
